In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
from pymongo import MongoClient

In [44]:
catlog = {}
for i in range(1, 60):
    resp = requests.get("https://tw.appledaily.com/new/realtime/1")
    soup = BeautifulSoup(resp.text, "html5lib")
    news_list = [[a.select("h2")[0].text, a.get("href")] for a in soup.select("div.abdominis > ul > li > a")]
    for news in news_list:
        if news[0] in catlog:
            catlog[news[0]].append(news[1])
        else:
            catlog[news[0]] = [news[1]]


In [45]:
catlog

{'副刊': ['https://tw.lifestyle.appledaily.com/lifestyle/realtime/20171129/1249495/',
  'https://tw.lifestyle.appledaily.com/lifestyle/realtime/20171129/1249495/',
  'https://tw.lifestyle.appledaily.com/lifestyle/realtime/20171129/1249495/',
  'https://tw.lifestyle.appledaily.com/lifestyle/realtime/20171129/1249495/',
  'https://tw.lifestyle.appledaily.com/lifestyle/realtime/20171129/1249495/',
  'https://tw.lifestyle.appledaily.com/lifestyle/realtime/20171129/1249495/',
  'https://tw.lifestyle.appledaily.com/lifestyle/realtime/20171129/1249495/',
  'https://tw.lifestyle.appledaily.com/lifestyle/realtime/20171129/1249495/',
  'https://tw.lifestyle.appledaily.com/lifestyle/realtime/20171129/1249495/',
  'https://tw.lifestyle.appledaily.com/lifestyle/realtime/20171129/1249495/',
  'https://tw.lifestyle.appledaily.com/lifestyle/realtime/20171129/1249495/',
  'https://tw.lifestyle.appledaily.com/lifestyle/realtime/20171129/1249495/',
  'https://tw.lifestyle.appledaily.com/lifestyle/realtime/

In [2]:
client = MongoClient('mongodb://localhost:27017/')
db = client["politics"]
collect = db["appledaily"]


In [47]:

for cat, urls in catlog.items():
    news_list = []
    for url in urls:
        news = {}
        resp = requests.get("https://tw.news.appledaily.com/international/realtime/20171129/1249870/")
        soup = BeautifulSoup(resp.text, "html5lib")
        news["title"] = soup.select("article.ndArticle_leftColumn > hgroup > h1")[0].text
        news["time"] = soup.select("article.ndArticle_leftColumn > hgroup > .ndArticle_creat")[0].text
        news["content"] = re.findall("(.+)（.+報導）", soup.select("article.ndArticle_content > div > p")[0].text)[0]
        news["category"] = cat
        news_list.append(news)
    collect.insert_many(news_list)

In [7]:
news = collect.find_one({}, {"_id": 0, "category": 1, "content": 1})

In [8]:
news

{'category': '社會',
 'content': '（更新：網友意見）澳洲墨爾本的弗林德斯街車站(Flinders Street Station)，在上周六晚間11時發生有兩名男子，當場在月台上做愛，不顧現場還有多位民眾。據英國《每日郵報》報導，目擊者指出，這兩名男子全身赤裸，當眾在月台上做愛並「磨來磨去」，相信現場還有數十名乘客都目睹這一切。警方指出這兩人分別為22歲與25歲，最後被控猥褻罪，另外兩人還偷了墨爾本市府公共盆栽，因此也被控竊盜罪。目擊者蘭奇克(Samantha Lenkic)就說，在現場她和其他數十名乘客都看到這一幕，還有一名阿嬤對他們吼叫制止，她也直呼：「我到底是看到了什麼...」她稱有人和她一樣，覺得驚恐，但也有圍觀者哈哈大笑。台灣網友紛紛表示：「蘋果日報第一次標題下得好～『無恥辱甲甲』！」、「畜牲無誤！」、「無恥男女比較好看。」、「來台灣辣！」'}